In [1]:
import pandas as pd
import numpy as np
import psycopg2
import requests
from tqdm import tqdm

In [2]:
path = 'https://almanak.mybluemix.net/chatbot/rules'

In [22]:
conn = psycopg2.connect(
    host="ec2-52-86-115-245.compute-1.amazonaws.com",
    database="d92em23mmlh07d",
    user="yjixuohawbxqpj",
    password="61a11b1ebe328f19fc17be47b54382fc78a89a613d3eaea7633ff4cf4a3d73ad"
)

cur = conn.cursor()

In [4]:
cur.execute(
    "select column_name from information_schema.columns where table_schema = 'public' and table_name = 'tb_regra' order by ordinal_position;"
)
columnsResult = cur.fetchall()
title = []
for i in columnsResult:
    title.append(i[0])

title

['id_regra',
 'id_jogo',
 'nm_regra',
 'nr_regra',
 'ds_regra',
 'fl_opcional',
 'dt_registro']

In [5]:
cur.execute(
    "select * from tb_regra order by 1;"
)

regras = cur.fetchall()

df = pd.DataFrame(regras)

for n,col in enumerate(title):
    df = df.rename(columns={n:col})

df.head(5)


,id_regra,id_jogo,nm_regra,nr_regra,ds_regra,fl_opcional,dt_registro
0,1,1,Principal,1,Uma pessoa de cada dupla joga um taco para fre...,nao,2022-01-01
1,2,1,Principal,2,Com a missão de aproximá-lo ao máximo que pude...,nao,2022-01-01
2,3,1,Principal,3,"O meio fio de uma rua, ou um muro.",nao,2022-01-01
3,4,1,Principal,4,Ganha essa a disputa o par do taco que aproxim...,nao,2022-01-01
4,5,1,Principal,5,"Se o bastão encostar ou passar do ponto, a dup...",nao,2022-01-01


In [17]:
insertList = []

for i in tqdm(range(len(df))):

    line = df.iloc[i]
    id = line.id_regra
    regra = line.ds_regra

    r = requests.post(path, regra).text

    if(id == 1 and 'Inicio' not in regra):
        r = r.replace('{"intents":[', '{"intents":["Inicio",')
    
    insert = "insert into tb_categoria_regra values(nextval('sq_categoria_regra'), "+  str(id) + ", " +"'"+ str(r)+"'"+", current_timestamp);"
    insertList.append(insert)
    

100%|██████████| 37/37 [00:49<00:00,  1.33s/it]


In [23]:
for i in tqdm(range(len(insertList))):
    l = insertList[i]
    cur.execute(l)

100%|██████████| 37/37 [00:07<00:00,  5.00it/s]


In [ ]:
cur.execute("refresh materialized view vw_categoria_regra;")

In [24]:
cur.execute("commit;")

In [25]:
cur.close
conn.close

<function connection.close>